In [1]:
import script as sc
import pandas as pd
import datetime as dt
from datetime import timedelta
import numpy as np
user = 'Yash'

In [50]:
sc.update_complete(user)

/Users/yashmehra/Google Drive/DATA 451/KineseItUp/script.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  observedData['index'] = range(len(observedData))


In [51]:
data = pd.DataFrame(sc.get_complete(user))
data.head()

,Unnamed: 0,Unnamed: 0.1,mean.vm,sd.vm,mean.ang,sd.ang,p625,dfreq,ratio.df,start.time,index,activity,coding,type
0,0,368947,1.027036,0.065807,-32.179828,8.090852,0.188766,4.102564,0.126940,2017-10-06 16:44:06,0,active,non-sedentary,H-1
1,1,368948,1.029727,0.136367,-32.955579,8.908046,0.222211,3.076923,0.174922,2017-10-06 16:44:07,1,active,non-sedentary,H-1
2,2,368949,1.003435,0.056664,-47.568056,6.244429,0.235925,3.076923,0.245706,2017-10-06 16:44:08,2,active,non-sedentary,H-1
3,3,368950,1.020198,0.070674,-54.486552,4.062847,0.479812,2.051282,0.241239,2017-10-06 16:44:09,3,active,non-sedentary,H-1
4,4,368951,1.003209,0.031647,-52.630436,2.084940,0.443848,2.051282,0.291673,2017-10-06 16:44:10,4,active,non-sedentary,H-1


In [52]:
data.shape

(114227, 14)

In [53]:
data['activity'].value_counts()

active                              63856
sitting still                       22444
sitting and typing                  13311
sitting with upper body movement     8550
lying                                5639
kneeling                              427
Name: activity, dtype: int64

In [83]:
data['type'].value_counts()

L-1     7321
E-8     7300
W-4     7244
A-6     7242
E-6     7241
H-2     7202
A-4     7202
E-14    7201
H-3     7139
L-3     7128
H-7     7119
H-1     7117
W-13    7111
E-7     7107
L-5     7107
A-8     7102
L-11    7053
W-9     6554
A-2     3061
Name: type, dtype: int64

In [2]:
struct = sc.load_struct()
struct['1']

{'AG': 'AG01-01.csv',
 'EndTime1': '2017-10-06 18:45:06',
 'EndTime2': '2017-10-03 20:47:15',
 'GT1': 'GTH01-01.csv',
 'GT2': 'GTL01-02.csv',
 'Raw': 'AG_AM01WRT (2017-10-02)RAW.csv',
 'StartTime1': '2017-10-06 16:44:06',
 'StartTime2': '2017-10-03 18:45:15',
 'WIDE': 'WIDE-01-01.csv',
 'skiplines1': '29515680',
 'skiplines2': '9946800'}

In [8]:
wide1 = pd.read_csv(sc.load_Data("AGG-Yash", "Wide01-01H.csv"))
gt1 = pd.read_csv(sc.load_Data(user, struct['1']['GT1']))
ag = pd.read_csv(sc.load_Data(user, struct['1']['AG']))

In [9]:
startTime1 = struct['1']['StartTime1']
endTime1 = struct['1']['EndTime1']
startTime1, endTime1

('2017-10-06 16:44:06', '2017-10-06 18:45:06')

In [10]:
wide1 = wide1[wide1.X1 <= endTime1]
wide1 = wide1[wide1.X1 >= startTime1]
ag = ag[ag['start.time'] >= startTime1]
ag = ag[ag['start.time'] <= endTime1]

In [11]:
len(wide1), len(gt1), len(ag)

(7194, 7265, 7261)

In [65]:
raw = pd.read_csv(sc.load_Data('fun', 'AG_AM01WRT (2017-10-02)RAW.csv'), skiprows=10, header=0)

In [19]:
raw_startTime = '2017-10-02 10:15:00'

In [27]:
raw_startTime, startTime1

('2017-10-02 10:15:00', '2017-10-06 16:44:06')

In [91]:
skiplines = 368946*80 
num_lines = 7265 * 80

In [72]:
raw = raw[raw.index > skiplines - 1]
raw = raw[raw.index < skiplines+num_lines]

In [73]:
raw.shape

(581200, 3)

In [93]:
skiplines

29515680

In [3]:
def subject_wide(subject, gt_num):
    raw = pd.read_csv(sc.load_Data("fun", subject["Raw"]), skiprows=10, header=0)
    print("Read raw")
    gt = pd.read_csv(sc.load_Data("Yash", subject['GT' + gt_num]))
    skiplines = int(subject['skiplines' + gt_num])
    num_lines = len(gt)*80
    raw = raw[raw.index > skiplines - 1]
    raw = raw[raw.index < skiplines+num_lines]
    print("Subset raw")
    acts = np.repeat(gt['activity'], 80)
    raw.reset_index()
    if(len(raw) != len(acts)):
        print(len(raw), len(acts))
        print("Houston we have a problem!")
    else:
        raw['activity'] = np.asarray(acts)
    return raw

In [4]:
def complete_wide(struct):
    complete = pd.DataFrame()
    for i in struct:
        print(i, ': 1')
        wide = subject_wide(struct[i], '1')
        tp = struct[i]['GT1']
        tp = tp[2:8]
        wide['type'] = tp
        complete = pd.concat([complete, wide])   
        if('GT2' in struct[i]):
            print(i, ': 2')
            wide = subject_wide(struct[i], '2')
            tp = struct[i]['GT2']
            tp = tp[2:8]
            wide['type'] = tp
            complete = pd.concat([complete, wide])    
    return complete

In [5]:
data = complete_wide(struct)
data.to_csv(sc.load_Data("AGG-Yash", "updated_wide.csv"))

1 : 1
Read raw
Subset raw
1 : 2
Read raw
Subset raw
3 : 1
Read raw
Subset raw
3 : 2
Read raw
Subset raw
4 : 1
Read raw
Subset raw
4 : 2
Read raw
Subset raw
5 : 1
Read raw
Subset raw
6 : 1
Read raw
Subset raw
6 : 2
Read raw
Subset raw
7 : 1
Read raw
Subset raw
7 : 2
Read raw
Subset raw
8 : 1
Read raw
Subset raw
8 : 2
Read raw
Subset raw
9 : 1
Read raw
Subset raw
13 : 1
Read raw
Subset raw
14 : 1
Read raw
Subset raw


In [43]:
data.head()

,Accelerometer X,Accelerometer Y,Accelerometer Z,activity,type
29515680,-0.605,0.617,0.070,active,H01-01
29515681,-0.621,0.602,0.023,active,H01-01
29515682,-0.605,0.590,-0.016,active,H01-01
29515683,-0.590,0.602,-0.043,active,H01-01
29515684,-0.594,0.648,-0.047,active,H01-01


In [49]:
data['activity'].value_counts()/80

active                              64312.0
sitting still                       22966.0
sitting and typing                  13337.0
sitting with upper body movement     8557.0
lying                                5651.0
Private / Not Coded                   840.0
kneeling                              427.0
Name: activity, dtype: float64

In [3]:
wide = pd.read_csv(sc.load_Data("AGG-Yash", "updated_wide.csv"))

In [4]:
wide.head()

,Unnamed: 0,Accelerometer X,Accelerometer Y,Accelerometer Z,activity,type
0,29515680,-0.605,0.617,0.070,active,H01-01
1,29515681,-0.621,0.602,0.023,active,H01-01
2,29515682,-0.605,0.590,-0.016,active,H01-01
3,29515683,-0.590,0.602,-0.043,active,H01-01
4,29515684,-0.594,0.648,-0.047,active,H01-01
